In [1]:
print("Hello World")

Hello World


In [1]:
from config.warehouse_config import get_db_engine
# Importació de llibreries necessàries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from services.eda import EDA  # Importa la classe EDA del fitxer EDA

# Carrega el DataFrame amb les dades
data = pd.read_sql('SELECT * FROM titles LIMIT 100', get_db_engine())
data.set_index('id', inplace=True)

# Crea una instància de la classe EDA
eda = EDA(data)

# 1. Relació entre variables
# Calcula la correlació entre variables numèriques
correlation_matrix = data.corr()

# Mostra la matriu de correlació com un mapa de calor
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Matriu de correlació')
plt.show()

# Gràfic de dispersió entre puntuació IMDb i popularitat TMDb
plt.figure(figsize=(8, 6))
sns.scatterplot(x='imdb_score', y='tmdb_popularity', data=data)
plt.title('Dispersió entre puntuació IMDb i popularitat TMDb')
plt.xlabel('Puntuació IMDb')
plt.ylabel('Popularitat TMDb')
plt.show()

# 2. Longevitat de les sèries
# Gràfic de dispersió entre durada de les sèries i popularitat TMDb
plt.figure(figsize=(8, 6))
sns.scatterplot(x='runtime', y='tmdb_popularity', data=data[data['type'] == 'series'])
plt.title('Dispersió entre durada de les sèries i popularitat TMDb')
plt.xlabel('Durada de les sèries (minuts)')
plt.ylabel('Popularitat TMDb')
plt.show()

# Gràfic de dispersió entre durada de les sèries i puntuació IMDb
plt.figure(figsize=(8, 6))
sns.scatterplot(x='runtime', y='imdb_score', data=data[data['type'] == 'series'])
plt.title('Dispersió entre durada de les sèries i puntuació IMDb')
plt.xlabel('Durada de les sèries (minuts)')
plt.ylabel('Puntuació IMDb')
plt.show()

# 3. Distribució temporal
# Agrupa les dades per any de llançament i compta el nombre de títols per any
titles_per_year = data.groupby('release_year').size()

# Plota la distribució de títols al llarg dels anys
plt.figure(figsize=(12, 6))
titles_per_year.plot(kind='line')
plt.title('Distribució de títols al llarg dels anys')
plt.xlabel('Any de llançament')
plt.ylabel('Nombre de títols')
plt.show()

# Agrupa les dades per dècada de llançament i compta el nombre de títols per dècada
data['release_decade'] = data['release_year'] // 10 * 10
titles_per_decade = data.groupby('release_decade').size()

# Plota la distribució de títols al llarg de les dècades
plt.figure(figsize=(12, 6))
titles_per_decade.plot(kind='line')
plt.title('Distribució de títols al llarg de les dècades')
plt.xlabel('Dècada de llançament')
plt.ylabel('Nombre de títols')
plt.show()

# 4. Segmentació de públic
# Agrupa les dades per gènere i calcula la mitjana de la puntuació IMDb i TMDb
genre_avg_scores = data.groupby('genre').agg({'imdb_score': 'mean', 'tmdb_popularity': 'mean'})

# Plota la mitjana de puntuació IMDb i TMDb per gènere
plt.figure(figsize=(12, 6))
genre_avg_scores.plot(kind='bar')
plt.title('Mitjana de puntuació IMDb i TMDb per gènere')
plt.xlabel('Gènere')
plt.ylabel('Mitjana')
plt.show()


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)